In [11]:
repo_url = 'https://github.com/ZZIDZZ/tensorflow-object-detection.git'
# Number of training steps - 1000 will train very quickly, but more steps will increase accuracy.
num_steps = 10000  # 200000 to improve

# Number of evaluation steps.
num_eval_steps = 50

MODELS_CONFIG = {
    'ssd_mobilenet_v2_keras': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'pipeline_file': 'pipeline.config',
        'batch_size': 12
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
        'batch_size': 12
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
        'batch_size': 8
    }
}

# Pick the model you want to use
# Select a model in `MODELS_CONFIG`.
selected_model = 'ssd_mobilenet_v2_keras'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/home/gamer/Desktop/machine-learning/models/research/pretrained_model'

In [2]:
!ls

asnu			    resize_images.py
data			    tensorflow_object_detection_training_colab.ipynb
deploy			    test
generate_tfrecord.py	    test.record
LICENSE			    tfrecodd
local_inference_test.ipynb  TRAININGTRAINING.ipynb
local_inference_test.py     TRAININGTRAININGlinux.ipynb
ppe			    train.record
README.md		    xml_to_csv.py
requirements.txt


In [2]:
%cd /home/gamer/Desktop/machine-learning/models/research
!protoc object_detection/protos/*.proto --python_out=.

import os
os.environ['PYTHONPATH'] += ':/home/gamer/Desktop/machine-learning/models/research/:/home/gamer/Desktop/machine-learning/models/research/slim/:/home/gamer/Desktop/machine-learning/models/'

!python object_detection/builders/model_builder_test.py

/home/gamer/Desktop/machine-learning/models/research


In [12]:
!python generate_tfrecord.py --csv_input data\annotations --img_path data\train

Traceback (most recent call last):
  File "d:\Projects\pyproj\tensorflow-object-detection\generate_tfrecord.py", line 134, in <module>
    tf.app.run()
  File "D:\Projects\pyproj\project_env\lib\site-packages\tensorflow\python\platform\app.py", line 36, in run
    _run(main=main, argv=argv, flags_parser=_parse_flags_tolerate_undef)
  File "D:\Projects\pyproj\project_env\lib\site-packages\absl\app.py", line 312, in run
    _run_main(main, args)
  File "D:\Projects\pyproj\project_env\lib\site-packages\absl\app.py", line 258, in _run_main
    sys.exit(main(argv))
  File "d:\Projects\pyproj\tensorflow-object-detection\generate_tfrecord.py", line 107, in main
    writer = tf.python_io.TFRecordWriter(FLAGS.output_path)
  File "D:\Projects\pyproj\project_env\lib\site-packages\tensorflow\python\lib\io\tf_record.py", line 294, in __init__
    super(TFRecordWriter, self).__init__(
tensorflow.python.framework.errors_impl.NotFoundError: Failed to create a NewWriteableFile:  : The system cannot fin

In [3]:
!which python

/home/gamer/Desktop/project_env/bin/python


In [3]:
%source /home/gamer/Desktop/machine-learning/activatepy.bashrc

UsageError: Line magic function `%source` not found.


In [4]:
%cd /home/gamer/Desktop/machine-learning/models/research

import os
import shutil
import glob
import urllib.request
import tarfile

MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/home/gamer/Desktop/machine-learning/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve('http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz', MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/home/gamer/Desktop/machine-learning/models/research


In [12]:
%cd /home/gamer/Desktop/machine-learning/models/research
fine_tune_checkpoint = os.path.join(DEST_DIR, "checkpoint/checkpoint")
fine_tune_checkpoint

/home/gamer/Desktop/machine-learning/models/research


'/home/gamer/Desktop/machine-learning/models/research/pretrained_model/checkpoint/checkpoint'

In [13]:
import os
pipeline_fname = os.path.join('/home/gamer/Desktop/machine-learning/models/research/object_detection/samples/configs/', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

AssertionError: `/home/gamer/Desktop/machine-learning/models/research/object_detection/samples/configs/pipeline.config` not exist

In [7]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [8]:
import re
train_record_fname = '/home/gamer/Desktop/machine-learning/tensorflow-object-detection/train.record'
test_record_fname = '/home/gamer/Desktop/machine-learning/tensorflow-object-detection/test.record'
label_map_pbtxt_fname = '/home/gamer/Desktop/machine-learning/tensorflow-object-detection/ppe/label_map.pbtxt'
num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    f.write(s)

In [26]:
!cat {pipeline_fname}

# SSD with Mobilenet v2 configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    num_classes: 3
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_

In [9]:
model_dir = 'training/'
# Optionally remove content in output model directory to fresh start.
!rm -rf {model_dir}
os.makedirs(model_dir, exist_ok=True)

In [28]:
!python3 /home/gamer/Desktop/machine-learning/models/research/object_detection/model_main.py --help

Binary to run train and evaluation on object detection model.
flags:

/home/gamer/Desktop/machine-learning/models/research/object_detection/model_main.py:
  --checkpoint_dir: Path to directory holding a checkpoint.  If `checkpoint_dir`
    is provided, this binary operates in eval-only mode, writing resulting
    metrics to `model_dir`.
  --[no]eval_training_data: If training data should be evaluated for this job.
    Note that one call only use this in eval-only mode, and `checkpoint_dir`
    must be supplied.
    (default: 'false')
  --max_eval_retries: If running continuous eval, the maximum number of retries
    upon encountering tf.errors.InvalidArgumentError. If negative, will always
    retry the evaluation.
    (default: '0')
    (an integer)
  --model_dir: Path to output model directory where event and checkpoint files
    will be written.
  --num_train_steps: Number of train steps.
    (an integer)
  --pipeline_config_path: Path to pipeline config file.
  --[no]run_once: If r

In [10]:
!python3 /home/gamer/Desktop/machine-learning/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

2022-02-25 00:07:36.882218: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-25 00:07:36.882674: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/gamer/Desktop/project_env/lib/python3.9/site-packages/cv2/../../lib64:
2022-02-25 00:07:36.882752: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/gamer/Desktop/project_env/lib/python3.9/site-packages/cv2/../../lib64:
2022-02-25 00:07:36.882809: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.s